In [1]:
import pandas as pd
import re
import nltk
import json
from collections import Counter

In [36]:
def compile_sql(sql):
    sql_stmts = []
    for i in sql:
        sql_stmts.append(re.sub("[^A-Za-z0-9\ =\[\]'#)\"\.+:-_{}*;/\\?=&]", "", i))
    return sql_stmts

def get_punkt(sql):
    temp = []
    for i in sql:
        temp.extend(re.findall("[^A-Za-z0-9\ =\[\]'#)\"\.+:-_{}*;/\\?=&]", i))
    temp = set(temp)
    return temp
    

In [16]:
sql_queries = pd.read_csv('../data/temp/nl_sql.csv', sep='\t')

In [23]:
punc = list(get_punkt(sql_queries["query"]))

In [34]:
sql_stmts = compile_sql(sql_queries["query"])

In [38]:
import random

In [64]:
nltk.word_tokenize("select tablealias0.format_field from table as tablealias0 where tablealias0.year_field < var0 ;")

['select',
 'tablealias0.format_field',
 'from',
 'table',
 'as',
 'tablealias0',
 'where',
 'tablealias0.year_field',
 '<',
 'var0',
 ';']

In [65]:
cnt = Counter()
for i in sql_stmts:
    cnt.update(nltk.word_tokenize(i))

In [68]:
cnt.most_common(10)

[('=', 61007),
 ('tablealias0', 51643),
 (';', 51176),
 ('from', 51170),
 ('select', 51159),
 ('table', 51159),
 ('as', 50981),
 ('where', 50762),
 ('var0', 50762),
 ('and', 27589)]

In [70]:
with open('../data/temp/sql.txt', "w+", encoding="utf-8") as f:
    for i in sql_stmts:
        f.write(f"{i}\n")

In [74]:
with open('../data/temp/sql.vocab', "w+", encoding="utf-8") as f:
    for i in cnt:
        f.write(f"{i}\t{cnt[i]}\n")

In [75]:
import _pickle as cPickle

In [77]:
def read_pickle(file):
    with open(file, 'rb') as f:
        return cPickle.load(f)

In [78]:
nlcnt = read_pickle("../data/temp/counter_nl.pickle")

In [79]:
with open('../data/temp/nl.vocab', "w+", encoding="utf-8") as f:
    for i in nlcnt:
        f.write(f"{i}\t{nlcnt[i]}\n")

In [83]:
fnltr, fnlte = open('../data/temp/nl-tr.txt', "w+", encoding="utf-8"), open('../data/temp/nl-val.txt', "w+", encoding="utf-8")
fsqltr, fsqlte = open('../data/temp/sql-tr.txt', "w+", encoding="utf-8"), open('../data/temp/sql-val.txt', "w+", encoding="utf-8")

In [86]:
for n, i in enumerate(zip(sql_queries["question"], sql_queries["query"])):
    if n < 35900:
        fnltr.write(f"{i[0]}\n")
        fsqltr.write(f"{i[1]}\n")
    else:
        fnlte.write(f"{i[0]}\n")
        fsqlte.write(f"{i[1]}\n")

In [87]:
fsqltr.close()
fsqlte.close()
fnltr.close()
fnlte.close()